Farmer problem (wait-and-see policy)

Mauricio Tombini and Victor M. Zavala 2016 

University of Wisconsin-Madison 

In [25]:
using JuMP 
using Ipopt 

In [26]:
# Model parameters

NS = 5;                    # number of scenarios
S = collect([1:NS]);       # scenario set
P = collect([1:3]);        # set of crops (1=wheat,2=corn,3=beets)

# Data 
prcost = zeros(3)          # production (planting) cost
prcost[1] = 150;
prcost[2] = 230;
prcost[3] = 260;

pcost = zeros(3)           # purchase cost
pcost[1] = 238;
pcost[2] = 210;
pcost[3] = 0;

scost = zeros(3)           # sales cost
scost[1] = 170;
scost[2] = 150;
scost[3] = 36;

demand = zeros(3)          # demand
demand[1] = 200;
demand[2] = 240;
demand[3] = 0;

# assign random data

yield = zeros(length(S),3) # yields
yield[S,1] = 2.5;
yield[S,2] = 3.0;

yield[1,3] = 10;
yield[2,3] = 15;
yield[3,3] = 20;
yield[4,3] = 25;
yield[5,3] = 30;

In this problem we remove the non-anticipativity constraints

In [27]:
# Model 

m = Model(solver=IpoptSolver(print_level=0))

@defVar(m, x[S,P] >= 0)    # acres devoted to crops
@defVar(m, y[S,P] >= 0)    # crops purchase
@defVar(m, w[S,P] >= 0)    # crops sold;
@defExpr(m, Cost[s in S], sum{prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j], j in P})
@defVar(m, cost[s in S])


@addConstraint(m, varcost[s in S], cost[s] == Cost[s]) 
@addConstraint(m, cap[s in S], sum{x[s,j], j in P} <= 500)
@addConstraint(m, bal[s in S,j in P], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j]) 
@addConstraint(m, sellb[s in S], w[s,3] <= 6000)
@addConstraint(m, buyb[s in S], y[s,3] <= 0)


@setObjective(m, Min, (1/NS)*sum{cost[s], s in S})

solve(m)

:Optimal

In [28]:
# Results

println(getValue(x))
println("")
println(getValue(y))
println("")
println(getValue(w))
println("")
println(getValue(cost))
println("")
println("obj: ", getObjectiveValue(m))

x: 2 dimensions:
[1,:]
  [1,1] = 420.0000058026925
  [1,2] = 79.99999920015239
  [1,3] = 0.0
[2,:]
  [2,1] = 79.99999920245911
  [2,2] = 79.99999920012897
  [2,3] = 340.00000659736713
[3,:]
  [3,1] = 120.00000280082533
  [3,2] = 79.99999920015239
  [3,3] = 300.0000029989767
[4,:]
  [4,1] = 180.00000340058205
  [4,2] = 79.99999920015239
  [4,3] = 240.00000239922
[5,:]
  [5,1] = 220.00000380042712
  [5,2] = 79.99999920015239
  [5,3] = 200.0000019993749


y: 2 dimensions:
[1,:]
  [1,1] = 0.0
  [1,2] = 0.0
  [1,3] = 9.752086119058693e-9
[2,:]
  [2,1] = 0.0
  [2,2] = 0.0
  [2,3] = 9.714578177618196e-9
[3,:]
  [3,1] = 0.0
  [3,2] = 0.0
  [3,3] = 9.580169952360632e-9
[4,:]
  [4,1] = 0.0
  [4,2] = 0.0
  [4,3] = 9.227902637280718e-9
[5,:]
  [5,1] = 0.0
  [5,2] = 0.0
  [5,3] = 8.765825074884802e-9


w: 2 dimensions:
[1,:]
  [1,1] = 850.0000164968418
  [1,2] = 0.0
  [1,3] = 0.0
[2,:]
  [2,1] = 0.0
  [2,2] = 0.0
  [2,3] = 5100.000098979874
[3,:]
  [3,1] = 100.00000899217387
  [3,2] = 0.0
  [3,3] =